In [242]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
from sklearn.svm import SVC
import csv

In [243]:
with open('gameSheet.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        print(row)

['', 'game_id', 'team', 'points', 'rebounds', 'assists', 'blocks', 'turnovers', 'steals', 'threes', 'winner', 'pt_dif', 'pg', 'sg', 'sf', 'pf', 'c']
['1', '71218-kings', 'kings', '60', '16', '18', '1', '4', '3', '4', '1', '2', '4', '5', '1', '1', '5']
['2', '71218-kings', 'warriors', '58', '21', '13', '3', '13', '8', '4', '0', '-2', '2', '5', '2', '2', '5']
['3', '71418-bucks', 'bucks', '55', '16', '17', '0', '12', '8', '2', '0', '-6', '4', '5', '5', '5', '2']
['4', '71418-bucks', 'pistons', '61', '15', '17', '4', '13', '9', '4', '1', '6', '4', '5', '2', '5', '2']
['5', '72018-kings', 'celtics', '77', '30', '28', '8', '6', '4', '8', '1', '5', '4', '5', '4', '5', '2']
['6', '72018-kings', 'kings', '72', '30', '21', '5', '4', '4', '8', '0', '-5', '4', '5', '3', '2', '5']
['7', '72118-mavs', 'grizz', '82', '11', '17', '2', '4', '6', '11', '1', '5', '2', '5', '1', '5', '4']
['8', '72118-mavs', 'mavs', '77', '15', '14', '1', '6', '4', '8', '0', '-5', '2', '5', '1', '5', '4']
['9', '72218-ma

In [244]:
def get_one_hot_encode(archetype):
    res = [0 for i in range(5)]
    res[archetype-1] = 1
    return res

In [260]:
gameSheet = []

with open('gameSheet.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        try:
            temp = []
            
            for i in range(3, len(row)):
                temp.append(int(row[i]))
            
            gameSheet.append(temp)
        
        except ValueError:
            continue
gameSheet = np.array(gameSheet)

In [298]:
x = []
y = []

for i in range(len(gameSheet)):
    temp = []
    if i % 2 == 0:
        for j in range(1, 5):
            temp.append(gameSheet[i][j] - gameSheet[i+1][j])
        x.append(temp)
        y.append(gameSheet[i][7])
    else:
        for j in range(1, 5):
            temp.append(gameSheet[i][j] - gameSheet[i-1][j])
        x.append(temp)
        y.append(gameSheet[i][7])

In [308]:
rf = SVC(kernel='linear')
rf.fit(x, y)
print(np.mean(cross_val_score(rf, x, y, cv=5)))
print(rf.coef_)

0.8666666666666666
[[ 0.08309625  0.87240853  0.43118298 -0.64137549]]


In [309]:
rf = RandomForestClassifier()
rf.fit(x, y)
print(np.mean(cross_val_score(rf, x, y, cv=5)))
print(rf.feature_importances_)

0.8333333333333333
[0.08356099 0.53775907 0.16071088 0.21796906]


In [314]:
with open('players.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        print(row)
        break

['game_id', 'team', 'player_id', 'matchup_id', 'position', 'archetype', 'field_goal_percentage', 'three_point_percentage', 'three_ptrs', 'rebounds', 'assists', 'steals', 'turnovers', 'blocks', 'points']


In [312]:
pgs = []
sgs = []
sfs = []
pfs = []
cs = []

with open('players.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row[4] == '1':
            pgs.append(row)
        elif row[4] == '2':
            sgs.append(row)
        elif row[4] == '3':
            sfs.append(row)
        elif row[4] == '4':
            pfs.append(row)
        elif row[4] == '5':
            cs.append(row)

In [323]:
pgs_defense = {}

for i in range(len(pgs)):
    if i % 2 == 0:
        matchup = pgs[i+1]
        if pgs[i][5] not in pgs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            pgs_defense[pgs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], pgs[i][11], pgs[i][13]]]
        else:
            pgs_defense[pgs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], pgs[i][11], pgs[i][13]])
    else:
        matchup = pgs[i-1]
        if pgs[i][5] not in pgs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            pgs_defense[pgs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], pgs[i][11], pgs[i][13]]]
        else:
            pgs_defense[pgs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], pgs[i][11], pgs[i][13]])

In [354]:
pgs_defense_avgs = {}

for key in pgs_defense:
    if key not in pgs_defense_avgs:
        pgs_defense_avgs[key] = {}
    for matchup in pgs_defense[key]:
        if matchup[0] not in pgs_defense_avgs[key]:
            pgs_defense_avgs[key][matchup[0]] = {}
            pgs_defense_avgs[key][matchup[0]]['tots'] = []
            pgs_defense_avgs[key][matchup[0]]['count'] = 1
            for i in range(1, len(matchup)):
                pgs_defense_avgs[key][matchup[0]]['tots'].append(float(matchup[i]))
        else:
            pgs_defense_avgs[key][matchup[0]]['count'] += 1
            for i in range(1, len(matchup)):
                pgs_defense_avgs[key][matchup[0]]['tots'][i-1] += float(matchup[i])


In [355]:
pgs_defense_avgs

{'2': {'2': {'count': 10, 'tots': [5.430000000000001, 28.0, 149.0, 23.0, 0.0]},
  '4': {'count': 4, 'tots': [1.3900000000000001, 11.0, 48.0, 8.0, 2.0]}},
 '4': {'2': {'count': 4, 'tots': [2.1246153846, 11.0, 84.0, 7.0, 1.0]},
  '4': {'count': 12, 'tots': [5.4750000000000005, 40.0, 184.0, 28.0, 1.0]}}}

In [371]:
print("POINT GUARDS")
for defender in pgs_defense_avgs:
    for matchup in pgs_defense_avgs[defender]:
        print(f"{defender} held {matchup} to {np.array(pgs_defense_avgs[defender][matchup]['tots']) / pgs_defense_avgs[defender][matchup]['count']}")
        
        
        
        
        
        
        
        

POINT GUARDS
4 held 2 to [ 0.53115385  2.75       21.          1.75        0.25      ]
4 held 4 to [ 0.45625     3.33333333 15.33333333  2.33333333  0.08333333]
2 held 4 to [ 0.3475  2.75   12.      2.      0.5   ]
2 held 2 to [ 0.543  2.8   14.9    2.3    0.   ]


In [362]:
sgs_defense = {}

for i in range(len(sgs)):
    if i % 2 == 0:
        matchup = sgs[i+1]
        if sgs[i][5] not in sgs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            sgs_defense[sgs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], sgs[i][11], sgs[i][13]]]
        else:
            sgs_defense[sgs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], sgs[i][11], sgs[i][13]])
    else:
        matchup = sgs[i-1]
        if sgs[i][5] not in sgs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            sgs_defense[sgs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], sgs[i][11], sgs[i][13]]]
        else:
            sgs_defense[sgs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], sgs[i][11], sgs[i][13]])

In [363]:
sgs_defense_avgs = {}

for key in sgs_defense:
    if key not in sgs_defense_avgs:
        sgs_defense_avgs[key] = {}
    for matchup in sgs_defense[key]:
        if matchup[0] not in sgs_defense_avgs[key]:
            sgs_defense_avgs[key][matchup[0]] = {}
            sgs_defense_avgs[key][matchup[0]]['tots'] = []
            sgs_defense_avgs[key][matchup[0]]['count'] = 1
            for i in range(1, len(matchup)):
                sgs_defense_avgs[key][matchup[0]]['tots'].append(float(matchup[i]))
        else:
            sgs_defense_avgs[key][matchup[0]]['count'] += 1
            for i in range(1, len(matchup)):
                sgs_defense_avgs[key][matchup[0]]['tots'][i-1] += float(matchup[i])


In [369]:
sgs_defense_avgs

{'2': {'5': {'count': 2, 'tots': [1.1, 1.0, 21.0, 0.0, 0.0]}},
 '5': {'2': {'count': 2, 'tots': [1.11, 0.0, 27.0, 1.0, 0.0]},
  '5': {'count': 26, 'tots': [15.203333333300002, 13.0, 263.0, 13.0, 0.0]}}}

In [370]:
print("SHOOTING GUARDS")
for defender in sgs_defense_avgs:
    for matchup in sgs_defense_avgs[defender]:
        print(f"{defender} held {matchup} to {np.array(sgs_defense_avgs[defender][matchup]['tots']) / sgs_defense_avgs[defender][matchup]['count']}")

SHOOTING GUARDS
5 held 5 to [ 0.58474359  0.5        10.11538462  0.5         0.        ]
5 held 2 to [ 0.555  0.    13.5    0.5    0.   ]
2 held 5 to [ 0.55  0.5  10.5   0.    0.  ]


In [372]:
sfs_defense = {}

for i in range(len(sfs)):
    if i % 2 == 0:
        matchup = sfs[i+1]
        if sfs[i][5] not in sfs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            sfs_defense[sfs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], sfs[i][11], sfs[i][13]]]
        else:
            sfs_defense[sfs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], sfs[i][11], sfs[i][13]])
    else:
        matchup = sfs[i-1]
        if sfs[i][5] not in sfs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            sfs_defense[sgs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], sfs[i][11], sfs[i][13]]]
        else:
            sfs_defense[sgs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], sfs[i][11], sfs[i][13]])

In [373]:
sfs_defense_avgs = {}

for key in sfs_defense:
    if key not in sfs_defense_avgs:
        sfs_defense_avgs[key] = {}
    for matchup in sfs_defense[key]:
        if matchup[0] not in sfs_defense_avgs[key]:
            sfs_defense_avgs[key][matchup[0]] = {}
            sfs_defense_avgs[key][matchup[0]]['tots'] = []
            sfs_defense_avgs[key][matchup[0]]['count'] = 1
            for i in range(1, len(matchup)):
                sfs_defense_avgs[key][matchup[0]]['tots'].append(float(matchup[i]))
        else:
            sfs_defense_avgs[key][matchup[0]]['count'] += 1
            for i in range(1, len(matchup)):
                sfs_defense_avgs[key][matchup[0]]['tots'][i-1] += float(matchup[i])


In [374]:
sfs_defense_avgs

{'1': {'1': {'count': 3, 'tots': [2.3600000000000003, 6.0, 78.0, 9.0, 2.0]},
  '2': {'count': 2, 'tots': [0.7914285714, 2.0, 14.0, 3.0, 0.0]}},
 '2': {'1': {'count': 1, 'tots': [0.71, 3.0, 25.0, 0.0, 0.0]},
  '4': {'count': 1, 'tots': [0.57, 3.0, 18.0, 0.0, 0.0]}},
 '3': {'1': {'count': 1, 'tots': [0.5, 0.0, 18.0, 0.0, 0.0]},
  '2': {'count': 2, 'tots': [1.38, 5.0, 24.0, 0.0, 0.0]},
  '3': {'count': 1, 'tots': [0.8, 0.0, 11.0, 0.0, 0.0]}},
 '4': {'1': {'count': 1, 'tots': [0.87, 5.0, 28.0, 1.0, 1.0]},
  '3': {'count': 3, 'tots': [1.9428571429000001, 1.0, 47.0, 4.0, 1.0]}},
 '5': {'1': {'count': 4, 'tots': [2.64, 17.0, 116.0, 7.0, 3.0]},
  '2': {'count': 1, 'tots': [0.64, 0.0, 21.0, 2.0, 0.0]},
  '3': {'count': 4, 'tots': [2.59, 1.0, 50.0, 1.0, 0.0]},
  '4': {'count': 3, 'tots': [1.4955555556000002, 6.0, 48.0, 4.0, 0.0]}}}

In [375]:
print("SMALL FORWARDS")
for defender in sfs_defense_avgs:
    for matchup in sfs_defense_avgs[defender]:
        print(f"{defender} held {matchup} to {np.array(sfs_defense_avgs[defender][matchup]['tots']) / sfs_defense_avgs[defender][matchup]['count']}")

SMALL FORWARDS
1 held 2 to [0.39571429 1.         7.         1.5        0.        ]
1 held 1 to [ 0.78666667  2.         26.          3.          0.66666667]
5 held 4 to [ 0.49851852  2.         16.          1.33333333  0.        ]
5 held 1 to [ 0.66  4.25 29.    1.75  0.75]
5 held 3 to [ 0.6475  0.25   12.5     0.25    0.    ]
5 held 2 to [ 0.64  0.   21.    2.    0.  ]
4 held 3 to [ 0.64761905  0.33333333 15.66666667  1.33333333  0.33333333]
4 held 1 to [ 0.87  5.   28.    1.    1.  ]
2 held 4 to [ 0.57  3.   18.    0.    0.  ]
2 held 1 to [ 0.71  3.   25.    0.    0.  ]
3 held 2 to [ 0.69  2.5  12.    0.    0.  ]
3 held 1 to [ 0.5  0.  18.   0.   0. ]
3 held 3 to [ 0.8  0.  11.   0.   0. ]


In [378]:
pfs_defense = {}

for i in range(len(pfs)):
    if i % 2 == 0:
        matchup = pfs[i+1]
        if pfs[i][5] not in pfs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            pfs_defense[pfs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], pfs[i][11], pfs[i][13]]]
        else:
            pfs_defense[pfs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], pfs[i][11], pfs[i][13]])
    else:
        matchup = pfs[i-1]
        if pfs[i][5] not in pfs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            pfs_defense[pfs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], pfs[i][11], pfs[i][13]]]
        else:
            pfs_defense[pfs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], pfs[i][11], pfs[i][13]])

In [379]:
pfs_defense_avgs = {}

for key in pfs_defense:
    if key not in pfs_defense_avgs:
        pfs_defense_avgs[key] = {}
    for matchup in pfs_defense[key]:
        if matchup[0] not in pfs_defense_avgs[key]:
            pfs_defense_avgs[key][matchup[0]] = {}
            pfs_defense_avgs[key][matchup[0]]['tots'] = []
            pfs_defense_avgs[key][matchup[0]]['count'] = 1
            for i in range(1, len(matchup)):
                pfs_defense_avgs[key][matchup[0]]['tots'].append(float(matchup[i]))
        else:
            pfs_defense_avgs[key][matchup[0]]['count'] += 1
            for i in range(1, len(matchup)):
                pfs_defense_avgs[key][matchup[0]]['tots'][i-1] += float(matchup[i])


In [380]:
pfs_defense_avgs

{'1': {'2': {'count': 1, 'tots': [0.64, 2.0, 19.0, 1.0, 2.0]},
  '5': {'count': 1, 'tots': [0.57, 0.0, 11.0, 0.0, 3.0]}},
 '2': {'1': {'count': 1, 'tots': [0.57, 1.0, 14.0, 0.0, 0.0]},
  '5': {'count': 3, 'tots': [2.16, 2.0, 24.0, 4.0, 4.0]}},
 '5': {'1': {'count': 1, 'tots': [0.43, 0.0, 6.0, 0.0, 1.0]},
  '2': {'count': 3, 'tots': [1.6954545455, 5.0, 36.0, 2.0, 7.0]},
  '5': {'count': 20, 'tots': [9.675, 18.0, 142.0, 19.0, 9.0]}}}

In [382]:
print("POWER FORWARDS")
for defender in pfs_defense_avgs:
    for matchup in pfs_defense_avgs[defender]:
        print(f"{defender} held {matchup} to {np.array(pfs_defense_avgs[defender][matchup]['tots']) / pfs_defense_avgs[defender][matchup]['count']}")

POWER FORWARDS
1 held 2 to [ 0.64  2.   19.    1.    2.  ]
1 held 5 to [ 0.57  0.   11.    0.    3.  ]
2 held 1 to [ 0.57  1.   14.    0.    0.  ]
2 held 5 to [0.72       0.66666667 8.         1.33333333 1.33333333]
5 held 5 to [0.48375 0.9     7.1     0.95    0.45   ]
5 held 2 to [ 0.56515152  1.66666667 12.          0.66666667  2.33333333]
5 held 1 to [0.43 0.   6.   0.   1.  ]


In [383]:
cs_defense = {}

for i in range(len(cs)):
    if i % 2 == 0:
        matchup = cs[i+1]
        if cs[i][5] not in cs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            cs_defense[cs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], cs[i][11], cs[i][13]]]
        else:
            cs_defense[cs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], cs[i][11], cs[i][13]])
    else:
        matchup = cs[i-1]
        if cs[i][5] not in cs_defense:
            # opp_archetype, opp_fg%, opp_turnovers, opp_points, pg_steals, pg_blocks
            cs_defense[cs[i][5]] = [[matchup[5], matchup[6], matchup[12], matchup[14], cs[i][11], cs[i][13]]]
        else:
            cs_defense[cs[i][5]].append([matchup[5], matchup[6], matchup[12], matchup[14], cs[i][11], cs[i][13]])

In [384]:
cs_defense_avgs = {}

for key in cs_defense:
    if key not in cs_defense_avgs:
        cs_defense_avgs[key] = {}
    for matchup in cs_defense[key]:
        if matchup[0] not in cs_defense_avgs[key]:
            cs_defense_avgs[key][matchup[0]] = {}
            cs_defense_avgs[key][matchup[0]]['tots'] = []
            cs_defense_avgs[key][matchup[0]]['count'] = 1
            for i in range(1, len(matchup)):
                cs_defense_avgs[key][matchup[0]]['tots'].append(float(matchup[i]))
        else:
            cs_defense_avgs[key][matchup[0]]['count'] += 1
            for i in range(1, len(matchup)):
                cs_defense_avgs[key][matchup[0]]['tots'][i-1] += float(matchup[i])


In [385]:
cs_defense_avgs

{'2': {'2': {'count': 16, 'tots': [11.52, 29.0, 322.0, 16.0, 32.0]},
  '3': {'count': 1, 'tots': [0.63, 1.0, 21.0, 0.0, 2.0]},
  '4': {'count': 2, 'tots': [1.0, 1.0, 15.0, 2.0, 1.0]},
  '5': {'count': 2, 'tots': [0.975, 2.0, 39.0, 1.0, 2.0]}},
 '3': {'2': {'count': 1, 'tots': [0.88, 2.0, 15.0, 1.0, 0.0]}},
 '4': {'2': {'count': 2, 'tots': [1.4, 4.0, 47.0, 0.0, 5.0]},
  '4': {'count': 2, 'tots': [1.66, 1.0, 29.0, 2.0, 1.0]}},
 '5': {'2': {'count': 2, 'tots': [1.2485714286, 1.0, 48.0, 1.0, 7.0]},
  '5': {'count': 2, 'tots': [1.0, 1.0, 14.0, 1.0, 4.0]}}}

In [386]:
print("CENTER")
for defender in cs_defense_avgs:
    for matchup in cs_defense_avgs[defender]:
        print(f"{defender} held {matchup} to {np.array(cs_defense_avgs[defender][matchup]['tots']) / cs_defense_avgs[defender][matchup]['count']}")

CENTER
5 held 5 to [0.5 0.5 7.  0.5 2. ]
5 held 2 to [ 0.62428571  0.5        24.          0.5         3.5       ]
2 held 2 to [ 0.72    1.8125 20.125   1.      2.    ]
2 held 5 to [ 0.4875  1.     19.5     0.5     1.    ]
2 held 4 to [0.5 0.5 7.5 1.  0.5]
2 held 3 to [ 0.63  1.   21.    0.    2.  ]
4 held 4 to [ 0.83  0.5  14.5   1.    0.5 ]
4 held 2 to [ 0.7  2.  23.5  0.   2.5]
3 held 2 to [ 0.88  2.   15.    1.    0.  ]
